In [1]:
import pandas as pd
import numpy as np
import os
os.environ['MKL_NUM_THREADS'] = '1'
from collections import Counter
import itertools
import json
import scipy.sparse as sparse
import pickle
import torch
from models.ae import implicitWrapper
import misc.util as util
from torch import nn

import importlib
from torch.utils.data import DataLoader
from misc.loader import AEDataset
from misc.util import *
from tqdm.auto import tqdm
from eval.rec_eval import *
import neuralsort.pl as pl
from models.loss import *
from misc.loader import RecDataset
import models
from neuralsort import neuralobjs
from torch.utils.tensorboard import SummaryWriter


In [2]:
dataset_name = "ml-20m"
with open(os.path.join("data", "parsed", dataset_name), 'rb') as f:
    a,b,c, tr_tr, tr_val, te = pickle.load(f)

In [3]:
n_users, n_items = tr_tr.shape

In [4]:
batch_sizes = [256, 512]
dims = [[512], [256], [128], [512, 256]]
dropouts = [0.5]
lambs = [0, 1e-5, 1e-4]
lrs = [1e-4, 5 * 1e-4, 1e-3, 3 * 1e-3]
n_epochs = 200

In [5]:
import itertools 
l = list(itertools.product(batch_sizes, dims, dropouts, lambs, lrs))
print(len(l))
#for x in l:
##    bs, dim, dropout, lamb, lr = x
#   print(x)

128


In [7]:
ae_dataset = AEDataset(tr_tr)
sc = neuralobjs.SC()

In [ ]:
for bs, dim, dropout, lamb, lr in itertools.product(batch_sizes, dims, dropouts, lambs, lrs):
    log_dir = os.path.join("logs/%s/" % dataset_name, "VAE", "%d-%s-%0.1f-%0.4f-%0.5f" %
                           (bs, '['+'_'.join([str(x) for x in dim])+']', dropout, lr, lamb)
                          )
    print(log_dir)
    writer = SummaryWriter(log_dir)
    loader = DataLoader(ae_dataset, batch_size=bs, shuffle=True, num_workers=6)
    vae = models.ae.MultiVAE(dim + [n_items], dropout=dropout).cuda()
    optimizer = torch.optim.RMSprop(vae.parameters(), lr=lr, weight_decay=lamb)    
    noupdate = 0 
    last_mp = -1
    for epoch in range(1, n_epochs + 1):
        tot_loss = 0
        vae.train()
        for uid, rowl in (loader):
            row = rowl.float().cuda()
            uid = uid.cuda() 
            scores, mu, logvar = vae.forward(row)
            loss  = models.ae.loss_function(scores, row, mu, logvar, anneal=1.0)
            loss.backward()
            optimizer.step()
        if (epoch % 5 == 0):
            vae.eval()
            wrapper = implicitWrapper(vae, tr_tr, tensor_wrapper=naive_sparse2tensor, vae=True)
            metric = ranking_metrics_at_k(wrapper, tr_tr, tr_val, K=30, num_threads=6)
            writer.add_scalar("val/precision", metric['precision'], epoch)
            writer.add_scalar("val/recall", metric['recall'], epoch)
            writer.add_scalar("val/ndcg", metric['ndcg'], epoch)
            writer.add_scalar("val/map", metric['map'], epoch)
            if metric['ndcg'] >= last_mp:
                noupdate = 0 
                last_mp = metric['ndcg']
                torch.save(
                    {"model": vae, 
                    "epoch": epoch,
                    "bs": bs,
                    "lr": lr,
                    "dim": dim, 
                    "lamb": lamb,
                    "dropout": dropout}, os.path.join("saved_models", "vae_%s" % dataset_name))
            else:
                noupdate += 1
        if noupdate >= 4:
            break
    writer.close()
    del vae

logs/ml-20m/VAE/256-[512]-0.5-0.0001-0.00000
logs/ml-20m/VAE/256-[512]-0.5-0.0005-0.00000
logs/ml-20m/VAE/256-[512]-0.5-0.0010-0.00000
logs/ml-20m/VAE/256-[512]-0.5-0.0030-0.00000
logs/ml-20m/VAE/256-[512]-0.5-0.0001-0.00100
logs/ml-20m/VAE/256-[512]-0.5-0.0005-0.00100
logs/ml-20m/VAE/256-[512]-0.5-0.0010-0.00100
logs/ml-20m/VAE/256-[512]-0.5-0.0030-0.00100
logs/ml-20m/VAE/256-[512]-0.5-0.0001-0.00300
logs/ml-20m/VAE/256-[512]-0.5-0.0005-0.00300
logs/ml-20m/VAE/256-[512]-0.5-0.0010-0.00300
logs/ml-20m/VAE/256-[512]-0.5-0.0030-0.00300
logs/ml-20m/VAE/256-[512]-0.5-0.0001-0.00500
logs/ml-20m/VAE/256-[512]-0.5-0.0005-0.00500
logs/ml-20m/VAE/256-[512]-0.5-0.0010-0.00500
logs/ml-20m/VAE/256-[512]-0.5-0.0030-0.00500
logs/ml-20m/VAE/256-[256]-0.5-0.0001-0.00000
logs/ml-20m/VAE/256-[256]-0.5-0.0005-0.00000
logs/ml-20m/VAE/256-[256]-0.5-0.0010-0.00000
logs/ml-20m/VAE/256-[256]-0.5-0.0030-0.00000
logs/ml-20m/VAE/256-[256]-0.5-0.0001-0.00100
logs/ml-20m/VAE/256-[256]-0.5-0.0005-0.00100
logs/ml-20

/home/ita/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1339: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [4]:
n_users, n_items = tr_tr.shape
dropout=0.5
import models
importlib.reload(models)
ae_dataset = AEDataset(tr_tr)
loader = DataLoader(ae_dataset, batch_size=512, shuffle=True, num_workers=6)
from neuralsort import neuralobjs
sc = neuralobjs.SC()
vae = models.ae.MultiVAE(
    [256, n_items],
    dropout=dropout).cuda()
lamb = 0.00
optimizer = torch.optim.RMSprop(vae.parameters(), lr=1 * 1e-3)
from scipy.sparse import csr_matrix
i = 0
mp = 0
for epoch in range(1, n_epochs + 1):
    tot_loss = 0
    tot_loss2 = 0 
    tot_loss3 = 0
    vae.train()

    for uid, rowl in (loader):
        row = rowl.float().cuda()
        uid = uid.cuda() 
        scores, mu, logvar = vae.forward(row)
        loss1  = models.ae.loss_function(scores, row, mu, logvar, anneal=1.0)
        loss2 = models.loss.neuPrecLoss(sc, scores, row, topk=100, k=10, tau=15.0).mean()
        (loss1 + 1e-3 * loss2).backward()
        optimizer.step()
        #print(loss1.mean().cpu().detach().numpy(), loss2.mean().detach().cpu().numpy())
    vae.eval()
    if (epoch % 5 == 0):
        wrapper = implicitWrapper(vae, tr_tr, tensor_wrapper=naive_sparse2tensor, vae=True)
        val_ndcg_10 = ranking_metrics_at_k(wrapper, tr_tr, tr_val, K=30, num_threads=6)
        mp = max(mp, val_ndcg_10['ndcg'])
        print(mp)


0.10477768227429503
0.10477768227429503
0.10477768227429503
0.10477768227429503
0.10477768227429503
0.10477768227429503


KeyboardInterrupt: 